<a href="https://colab.research.google.com/github/idocarmo/LAB6/blob/develop/frequency_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Frequency Test
## Validating the Bit Array Randomness 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
FOLDER_PATH = r"/content/drive/MyDrive/LAB6/data/02_processed/"

In [4]:
BACKGROUND_FILE = "bit_fundo.csv"
CESIUM_25MM_FILE = "bit_25mm_cs.csv"
CESIUM_65MM_FILE = "bit_65mm_cs.csv"

In [10]:
bit_background = np.loadtxt(FOLDER_PATH + BACKGROUND_FILE, delimiter=" ", dtype=int)

In [11]:
bit_25mm_cs = np.loadtxt(FOLDER_PATH + CESIUM_25MM_FILE, delimiter=" ", dtype=int)

In [12]:
bit_65mm_cs = np.loadtxt(FOLDER_PATH + CESIUM_65MM_FILE, delimiter=" ", dtype=int)

In [13]:
def erfc(s):
    """Complementary Error Function
    """
    def integrand(u):
        return (2/np.sqrt(np.pi))*np.exp(-u**2)

    return quad(integrand, s, np.inf)[0]

In [14]:
def frequency_test(bit_array: np.array, n_bit_streams: int=2) -> np.array:
    bit_array = bit_array.copy()

    bit_array = bit_array.reshape(n_bit_streams, -1)

    pvalue_array = np.zeros(n_bit_streams, dtype=float)

    for i, bit_stream in enumerate(bit_array):
        S = abs(sum(bit_stream==1) - sum(bit_stream==0))
        s_obs = S/np.sqrt(bit_stream.size)
        
        pvalue = erfc(s_obs/np.sqrt(2))

        pvalue_array[i] = pvalue

    return pvalue_array

In [21]:
sum(frequency_test(bit_background, n_bit_streams=10)>0.01)

10

In [20]:
sum(frequency_test(bit_25mm_cs, n_bit_streams=1000)>0.01)

1000

In [17]:
sum(frequency_test(bit_65mm_cs, n_bit_streams=1000)>0.01)

1000